In [114]:
import pandas as pd
import numpy as np 
from util import get_split_cols, get_split_frame
from sklearn.preprocessing import StandardScaler

In [462]:
train = pd.read_csv("/data/project2/train.csv")

test = pd.read_csv("/data/project2/test.csv")

In [463]:
num_train_cols, cat_train_cols = get_split_cols(train)
num_test_cols, cat_test_cols = get_split_cols(test)

In [464]:
assert(len(num_train_cols)==len(num_test_cols))
assert(len(cat_train_cols)==len(cat_test_cols))


In [465]:
num_train_df, cat_train_df = get_split_frame(train)
num_test_df, cat_test_df = get_split_frame(test)

In [573]:
y = num_train_df['Total Household Income']
num_train_df = num_train_df.drop(["Total Household Income"], axis=1)
len(num_train_df)

33235

In [544]:
class Layer(object):
    def __init__(self, num_nodes, num_node_in_prev_layer, eta, activation='relu'):
        self.output = np.ones(num_nodes)
        self.weights = np.random.randn(num_nodes, num_node_in_prev_layer)
#         print(self.weights)
#         print("dim", self.weights.shape)
        self.deltas = np.ones(num_nodes)
        self.activation_type = activation
        self.eta = eta
        
    def forward(self, inputs, no_act=False):
#         print("inputs", inputs)
#         print("weights", self.weights)
        self.inputs = inputs
        self.sum_w_h = np.dot(self.weights, self.inputs)
#         print("sum: ", self.sum_w_h)
        self.output = self.activation(self.sum_w_h)
        #print("out: ", self.output)
        
    def activation(self, sum_w_h):
        if self.activation_type == "sigmoid":
            return 1 / (1 + np.exp(-sum_w_h))
        if self.activation_type == "relu":
            return np.maximum(sum_w_h, 0)
        return sum_w_h
    
    def activation_prime(self, fx):
        if self.activation_type == "sigmoid": 
            return fx*(1 - fx)
        if self.activation_type == "relu":
            return np.sign(fx)
        return -100
        
    def backward(self, prev_deltas):
#         print("prev: ", prev_deltas)
#         print("weights: ", self.weights.T)
#         print("inputs T: ", self.inputs.T)
#         print("output: ", self.output)
#         print("sum: ", self.sum_w_h)
        if self.activation_type != None:
            a_prime = self.activation_prime(self.output)
#             print("a prime: ",  a_prime)
            self.delta = np.dot(prev_deltas * a_prime, self.inputs.T)
            
#             print("delta: ", self.delta)
            self.next_delta = np.dot(self.weights.T, prev_deltas * a_prime)
#             print("next: ", self.next_delta)
        else:
            
            #print("inputs: ", self.inputs)
            #print(type(self.inputs))

            self.delta = np.dot(prev_deltas, self.inputs.T)
            self.next_delta = np.dot(self.weights.T, prev_deltas)
            
#         print("delta: ", self.next_delta)
#         print("next: ", self.next_delta)
        self.update()


            
    def update(self):
        #print("deltas", self.delta)
        #print("change:", self.weights - self.eta*self.delta)
        self.weights = self.weights - self.eta*self.delta
        #print(self.weights)
#             self.deltas = 
#             self.deltas = self.weights * next_deltas
#             self.weights -= self.eta * self.inputs.T * next_deltas 
#         else:
#             self.deltas = self.weights * next_deltas
#             self.deltas = self.deltas * self.d_activation(self.i_w)
#             self.weights -= self.eta * np.matrix(self.inputs).T * self.d_activation(self.i_w) * next_deltas

class NeuralNetwork(object):
    
    def __init__(self, num_hidden_layers, num_nodes_per_layer, num_inputs, num_outputs, eta=0.00001, act='relu'):
        #assert(num_layers == len(num_nodes_per_layer))
        #num_nodes_per_layer = [num_inputs] + num_nodes_per_layer + [num_outputs]
        #print(num_nodes_per_layer)
        #self.layers = [Layer(num_nodes_per_layer[i], num_nodes_per_layer[i-1], eta, act) for i in range(1, num_layers+2)]
        self.network = [Layer(num_nodes=num_nodes_per_layer, num_node_in_prev_layer=num_inputs, eta=eta, activation=act)]
        self.network += [Layer(num_nodes=num_nodes_per_layer, num_node_in_prev_layer=num_nodes_per_layer, eta=eta, activation=act) for _ in range(num_hidden_layers - 1)]
        self.network += [Layer(num_nodes=num_outputs, num_node_in_prev_layer=num_nodes_per_layer,eta=eta, activation=None)]
        #print(len(self.network))
        self.eta = eta 
        
    def forward(self, x):
        for layer in self.network:
            layer.forward(x)
            x = layer.output
            #print("x", x)
        #print("outputsss: ", self.network[1].output)
            #print(self.network[1].output.shape)
        #self.layers[-1].forward(x, True)
            
    def backward(self, expected):
        #print(self.network[-1].output)
        deltas = self.network[-1].output - expected
#         self.layers[-1].backward(deltas, True)
#         deltas = self.layers[-1].deltas
        for layer in reversed(self.network):
            layer.backward(deltas)
            #layer.update()
            deltas = layer.next_delta
            
            
    def train(self, x, y, epoch):
        prev_err = 0 
        for i in range(epoch):
            self.sum_error = 0 
            count = 0
            for xi, yi in zip(x,y):
                #print(np.array([xi]).T)
                #return
                self.forward(np.array([xi]).T)
                self.backward(np.array([yi]))
                #self.update()
                #print(self.network[-1].output, yi)
                self.sum_error += (self.network[-1].output - yi)**2
                #print(i)
                #if count%10000 == 0:
                #    print("Sum Error: ", self.sum_error)    
                count += 1
            print("Sum Error: ", self.sum_error)    


            
            
    def predict(self, x):
        self.forward(x)
        return self.network[-1].output

In [547]:
# l = Layer(1,3, 0.005, None)
# l.forward(np.array([2,2,2]))
# l.backward(np.array([1,1,1]), [6])
# l.update()
data = np.array([[1, 1],[2, 2],[3, 3],[4, 4],[5, 5],[6, 6],[7, 7],[8, 8],[9, 9],[10, 10],[11, 11],[12, 12],[13, 13],[14, 14],[15, 15],[16, 16],[17, 17],[18, 18],[19, 19],[20, 20],[21, 21],[22, 22],[23, 23],[24, 24],[25, 25],[26, 26],[27, 27],[28, 28],[29, 29],[30, 30],[31, 31],[32, 32],[33, 33],[34, 34],[35, 35],[36, 36],[37, 37],[38, 38],[39, 39],[40, 40],[41, 41],[42, 42],[43, 43],[44, 44],[45, 45],[46, 46],[47, 47],[48, 48],[49, 49],[50, 50],[51, 51],[52, 52],[53, 53],[54, 54],[55, 55],[56, 56],[57, 57],[58, 58],[59, 59],[60, 60],[61, 61],[62, 62],[63, 63],[64, 64],[65, 65],[66, 66],[67, 67],[68, 68],[69, 69],[70, 70],[71, 71],[72, 72],[73, 73],[74, 74],[75, 75],[76, 76],[77, 77],[78, 78],[79, 79],[80, 80],[81, 81],[82, 82],[83, 83],[84, 84],[85, 85],[86, 86],[87, 87],[88, 88],[89, 89],[90, 90],[91, 91],[92, 92],[93, 93],[94, 94],[95, 95],[96, 96],[97, 97],[98, 98],[99, 99],[100, 100]])
expected = np.array([2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200])
nn = NeuralNetwork(1, 3, 2, 1)
data.T
#nn.forward(data.T)
#nn.backward(expected)
nn.train(data, expected, 4)
nn.predict(np.array([[3,2]]).T)

Sum Error:  [[ 48434.64205698]]
Sum Error:  [[  1.91511762e-26]]
Sum Error:  [[  1.91511762e-26]]
Sum Error:  [[  1.91511762e-26]]


array([[ 3.19064118]])

In [548]:
nn.predict(np.array([[3,6]]).T)

array([[ 15.25828758]])

In [549]:
data = np.array([[1, 1]])#,[2, 2],[3, 3],[4, 4],[5, 5],[6, 6],[7, 7],[8, 8],[9, 9],[10, 10],[11, 11],[12, 12],[13, 13],[14, 14],[15, 15],[16, 16],[17, 17],[18, 18],[19, 19],[20, 20],[21, 21],[22, 22],[23, 23],[24, 24],[25, 25],[26, 26],[27, 27],[28, 28],[29, 29],[30, 30],[31, 31],[32, 32],[33, 33],[34, 34],[35, 35],[36, 36],[37, 37],[38, 38],[39, 39],[40, 40],[41, 41],[42, 42],[43, 43],[44, 44],[45, 45],[46, 46],[47, 47],[48, 48],[49, 49],[50, 50],[51, 51],[52, 52],[53, 53],[54, 54],[55, 55],[56, 56],[57, 57],[58, 58],[59, 59],[60, 60],[61, 61],[62, 62],[63, 63],[64, 64],[65, 65],[66, 66],[67, 67],[68, 68],[69, 69],[70, 70],[71, 71],[72, 72],[73, 73],[74, 74],[75, 75],[76, 76],[77, 77],[78, 78],[79, 79],[80, 80],[81, 81],[82, 82],[83, 83],[84, 84],[85, 85],[86, 86],[87, 87],[88, 88],[89, 89],[90, 90],[91, 91],[92, 92],[93, 93],[94, 94],[95, 95],[96, 96],[97, 97],[98, 98],[99, 99],[100, 100]])
expected = np.array([[2]])#,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200])

In [593]:
scalar_x = StandardScaler()
scalar_y = StandardScaler()

x = scalar_x.fit_transform(num_train_df.values)
yy = scalar_y.fit_transform(y.values.reshape(-1,1))
#yy =  y/1000000.0
#inputs = len(num_train_df.columns)

#num_nodes = [inputs] * 1 

#n = NeuralNetwork(num_layers = 1, num_nodes_per_layer= num_nodes, num_inputs=inputs,
#                  num_outputs=1, eta=.00005, act='relu')
#n.train(x, yy, 100)
nn = NeuralNetwork(1, 46, 45, 1, .0001, "relu")
data.T
#nn.forward(data.T)
#nn.backward(expected)
nn.train(x, yy, 20)
nn.predict(np.array([x[0]]).T)

/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Sum Error:  [[ 1359070.94910271]]
Sum Error:  [[ 20241.90607037]]
Sum Error:  [[ 15975.25070595]]
Sum Error:  [[ 14793.62425144]]
Sum Error:  [[ 14203.67313629]]
Sum Error:  [[ 13824.53932989]]
Sum Error:  [[ 13540.48178955]]
Sum Error:  [[ 13306.85818335]]
Sum Error:  [[ 13103.19580359]]
Sum Error:  [[ 12921.29494061]]
Sum Error:  [[ 12755.85591777]]
Sum Error:  [[ 12602.52085026]]
Sum Error:  [[ 12459.60697202]]
Sum Error:  [[ 12325.89716661]]
Sum Error:  [[ 12199.8541182]]
Sum Error:  [[ 12082.20262054]]
Sum Error:  [[ 11965.30860911]]
Sum Error:  [[ 11854.74793299]]
Sum Error:  [[ 11754.43701322]]
Sum Error:  [[ 11667.1423213]]


array([[-0.14554512]])

In [1075]:
print(scalar_y.inverse_transform([(500.31)**.5]))
print(scalar_y.inverse_transform(n.predict(x[0])))
#print(n.predict(x[0]) * 1000000)
print(y[0])


[ 6664725.57242802]
[[ 204057.56275632]]
115835


In [577]:
x[0], yy[0]

(array([ 0.15690151, -0.06041136,  0.84197899,  1.40905177,  1.87447969,
        -0.1435758 , -0.76934503,  0.88236583,  0.49253331, -0.55458203,
        -0.43245302, -0.57065945, -0.41820001, -0.49833125, -0.44738637,
        -0.24877036, -0.49785752, -0.47470829, -0.3570236 , -0.39739303,
        -0.00952989,  0.19745053, -0.05610169,  0.82332918,  0.59968097,
         2.28583399, -0.25800044,  0.6347589 ,  5.25789187,  0.69847598,
         0.19338255,  0.34924226, -1.32532542, -0.77362673, -0.41673092,
        -0.7357341 ,  1.41790101, -0.28881982, -0.23192331, -0.21710951,
         0.05967282, -0.42682999, -0.38439565, -0.10374799, -0.52050519]),
 array([-0.45803026]))